In [1]:
using Pkg
Pkg.activate("..")

  Activating project at `/lustre/eaglefs/projects/pvb/cju/ProgressiveHedging.jl`


In [2]:
function print_struct(type)
    mutable = ismutable(type) ? "mutable" : ""
    println("$mutable struct $type")
    for (fn, ft) in zip(fieldnames(type), fieldtypes(type))
        println("    $fn::$ft")
    end
    println("end")
end

using AbstractTrees

In [3]:
using Distributed
const WORKERS = 1 # Change to > 1 to use parallel
if nworkers() < WORKERS
    diff = (nprocs() == nworkers() ? WORKERS : WORKERS - nworkers())
    println("Adding $diff worker processes.")
    Distributed.addprocs(diff)
    # Make sure these workers also have an environment with PH installed
    @everywhere using Pkg
    for w in workers()
        @spawnat(w, Pkg.activate(".."))
    end
end

@everywhere using ProgressiveHedging
@everywhere const PH = ProgressiveHedging
@everywhere using Ipopt
@everywhere using JuMP, Xpress, PowerSimulations, PowerSystems, PowerSystemCaseBuilder, TimeSeries
@everywhere const PSI = PowerSimulations
@everywhere const PSY = PowerSystems
@everywhere const PSB = PowerSystemCaseBuilder

┌ Info: Xpress: Found license file /nopt/nrel/apps/xpressmp/8.13.0/bin/xpauth.xpr
└ @ Xpress /home/cju/.julia/packages/Xpress/xOQbX/src/license.jl:44
┌ Info: Xpress: Development license detected.
└ @ Xpress /home/cju/.julia/packages/Xpress/xOQbX/src/license.jl:89


In [4]:
using JuMP, Xpress, PowerSimulations, PowerSystems, PowerSystemCaseBuilder, InfrastructureSystems, Dates, TimeSeries
const PSI = PowerSimulations
const PSY = PowerSystems
const PSB = PowerSystemCaseBuilder
solver = optimizer_with_attributes(Xpress.Optimizer, "MIPRELSTOP" => 1e-4)

MathOptInterface.OptimizerWithAttributes(Xpress.Optimizer, Pair{MathOptInterface.AbstractOptimizerAttribute, Any}[MathOptInterface.RawOptimizerAttribute("MIPRELSTOP") => 0.0001])

In [5]:
system = PSB.build_system(SIIPExampleSystems, "5_bus_hydro_uc_sys"; force_build =true);

┌ Info: Building new system 5_bus_hydro_uc_sys from raw data
│   sys_descriptor.raw_data = /home/cju/.julia/packages/PowerSystemCaseBuilder/WBLmn/data
└ @ PowerSystemCaseBuilder /home/cju/.julia/packages/PowerSystemCaseBuilder/WBLmn/src/build_system.jl:38
┌ Info: Parsing csv data in Hydro_Upstream_Input.csv ...
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/parsers/power_system_table_data.jl:144
┌ Info: Successfully parsed Hydro_Upstream_Input.csv
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/parsers/power_system_table_data.jl:149
┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/parsers/power_system_table_data.jl:144
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/parsers/power_system_table_data.jl:149
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /home/cju/.julia/packages/PowerSystems/sxHdO/src/parsers/power_system_table

In [6]:
get_name.(collect(get_components(ThermalStandard, system)))

5-element Vector{String}:
 "Solitude"
 "Park City"
 "Alta"
 "Brighton"
 "Sundance"

### Replicate what Sourabh queried ("S2.ipynb")

In [7]:
get(system.data.components.data, ThermalStandard, nothing)

Dict{String, ThermalStandard} with 5 entries:
  "Solitude"  => ThermalStandard(Solitude, true, true, Bus(3, bus3, BusTypes.PV…
  "Park City" => ThermalStandard(Park City, true, true, Bus(1, bus1, BusTypes.P…
  "Alta"      => ThermalStandard(Alta, true, true, Bus(1, bus1, BusTypes.PV = 3…
  "Brighton"  => ThermalStandard(Brighton, true, true, Bus(10, bus5, BusTypes.P…
  "Sundance"  => ThermalStandard(Sundance, true, true, Bus(4, bus4, BusTypes.RE…

In [8]:
# print_tree(system, depth=1)

In [9]:
# print_struct(Bus)

In [10]:
print_struct(typeof(system.data.components))
system.data.components.data

mutable struct InfrastructureSystems.Components
    data::Dict{DataType, Dict{String, <:InfrastructureSystems.InfrastructureSystemsComponent}}
    time_series_storage::InfrastructureSystems.TimeSeriesStorage
    validation_descriptors::Vector
end


Dict{DataType, Dict{String, <:InfrastructureSystems.InfrastructureSystemsComponent}} with 12 entries:
  Area                       => Dict{String, Area}("1"=>Area(1, 0.0, 0.0, 0.0),…
  HydroEnergyReservoir       => Dict{String, HydroEnergyReservoir}("HydroDispat…
  Bus                        => Dict{String, Bus}("bus1"=>Bus(1, bus1, BusTypes…
  PowerLoad                  => Dict{String, PowerLoad}("bus4"=>PowerLoad(bus4,…
  ThermalStandard            => Dict{String, ThermalStandard}("Solitude"=>Therm…
  Line                       => Dict{String, Line}("branch2"=>Line(branch2, tru…
  HydroPumpedStorage         => Dict{String, HydroPumpedStorage}("PHES"=>HydroP…
  TapTransformer             => Dict{String, TapTransformer}("branch5"=>TapTran…
  LoadZone                   => Dict{String, LoadZone}("1"=>LoadZone(1, 0.28571…
  Arc                        => Dict{String, Arc}("bus2 -> bus3"=>Arc(Bus(2, bu…
  VariableReserve{ReserveUp} => Dict{String, VariableReserve{ReserveUp}}("REG1"…
  Hydro

Below, we can get the ThreePartCost and the Dictionary mapping Key to Value, but we can't get both.

In [11]:
get_operation_cost.(collect(get_components(ThermalStandard, system)))

5-element Vector{ThreePartCost}:
 ThreePartCost(VariableCost{Vector{Tuple{Float64, Float64}}}([(0.0, 0.0), (525.0, 17.5), (1137.5, 35.0)]), 0.0, 0.0, 0.0)
 ThreePartCost(VariableCost{Vector{Tuple{Float64, Float64}}}([(0.0, 0.0), (165.0, 11.0)]), 0.0, 0.0, 0.0)
 ThreePartCost(VariableCost{Vector{Tuple{Float64, Float64}}}([(0.0, 0.0), (36.4, 2.6)]), 0.0, 0.0, 0.0)
 ThreePartCost(VariableCost{Vector{Tuple{Float64, Float64}}}([(0.0, 0.0), (400.0, 40.0)]), 0.0, 0.0, 0.0)
 ThreePartCost(VariableCost{Vector{Tuple{Float64, Float64}}}([(0.0, 0.0), (8000.0, 200.0)]), 0.0, 0.0, 0.0)

In [12]:
system.data.components.data[ThermalStandard]

Dict{String, ThermalStandard} with 5 entries:
  "Solitude"  => ThermalStandard(Solitude, true, true, Bus(3, bus3, BusTypes.PV…
  "Park City" => ThermalStandard(Park City, true, true, Bus(1, bus1, BusTypes.P…
  "Alta"      => ThermalStandard(Alta, true, true, Bus(1, bus1, BusTypes.PV = 3…
  "Brighton"  => ThermalStandard(Brighton, true, true, Bus(10, bus5, BusTypes.P…
  "Sundance"  => ThermalStandard(Sundance, true, true, Bus(4, bus4, BusTypes.RE…

In [13]:
(system.data.components.data[ThermalStandard])["Solitude"]

Solitude (ThermalStandard):
   name: Solitude
   available: true
   status: true
   bus: bus3 (Bus)
   active_power: 0.6285714285714286
   reactive_power: 0.7428571428571429
   rating: 1.2457273918052367
   active_power_limits: (min = 0.4857142857142857, max = 1.0)
   reactive_power_limits: (min = -0.7428571428571429, max = 0.7428571428571429)
   ramp_limits: (up = 0.42857142857142855, down = 0.42857142857142855)
   operation_cost: ThreePartCost
   base_power: 35.0
   time_limits: (up = 4.0, down = 4.0)
   prime_mover: PrimeMovers.CT = 8
   fuel: ThermalFuels.NATURAL_GAS = 7
   services: 0-element Vector{Service}
   time_at_status: 10000.0
   dynamic_injector: nothing
   ext: Dict{String, Any}()
   time_series_container: InfrastructureSystems.TimeSeriesContainer: 0
   InfrastructureSystems.SystemUnitsSettings:
      base_value: 100.0
      unit_system: UnitSystem.DEVICE_BASE = 1

In [14]:
for th in get_components(ThermalStandard, system)
    set_operation_cost!(th, new_cost[th.name])
end


LoadError: UndefVarError: new_cost not defined

In [59]:
for key in keys(system.data.components.data)
    println(key)
    for comp in get_components(key, system)
        println(comp)
    end
    println(" <<<<<<<<<<<<<<<<<<<<")
end

Area
Area(1, 0.0, 0.0, 0.0)
Area(2, 0.0, 0.0, 0.0)
 <<<<<<<<<<<<<<<<<<<<
HydroEnergyReservoir
HydroEnergyReservoir(HydroDispatch1, true, Bus(1, bus1, BusTypes.PV = 3, 0.0008540771836226905, 1.0, (min = 0.95, max = 1.05), 230.0, Area(1, 0.0, 0.0, 0.0), LoadZone(1, 0.285714286, 0.09390714300000001), Dict{String, Any}()), 0.0, 0.0, 0.07071067811865477, PrimeMovers.HY = 16, (min = 0.0, max = 0.05), (min = -0.05, max = 0.05), (up = 0.05, down = 0.05), nothing, 5.0, 7.686, 0.05, 7.686, TwoPartCost(VariableCost{Float64}(0.15), 0.0), 1.0, 1.0, 10000.0, Service[VariableReserve(REG1, true, 5.0, 0.001, Dict{String, Any}())], nothing, Dict{String, Any}())
HydroEnergyReservoir(HydroDispatch3, true, Bus(10, bus5, BusTypes.PV = 3, 0.0010936770654406082, 1.0, (min = 0.95, max = 1.05), 230.0, Area(1, 0.0, 0.0, 0.0), LoadZone(1, 0.285714286, 0.09390714300000001), Dict{String, Any}()), 0.0, 0.0, 1.0606601717798214, PrimeMovers.HY = 16, (min = 0.0, max = 0.75), (min = -0.75, max = 0.75), (up = 0.05, down 

### Let's get the optimization problem

In [16]:
template_ed = ProblemTemplate(CopperPlatePowerModel)
set_device_model!(template_ed, ThermalStandard, ThermalDispatchNoMin)
set_device_model!(template_ed, PowerLoad, StaticPowerLoad)
set_device_model!(template_ed, HydroDispatch, FixedOutput)
set_device_model!(template_ed, HydroEnergyReservoir, HydroDispatchRunOfRiver)

In [20]:
problem = DecisionModel(
        template_ed, 
        system, 
        name = "SubProblem",
        optimizer = solver, 
        # balance_slack_variables = true, # maybe this is depreciated
        warm_start = false,
        export_pwl_vars = true,
#         horizon = 2,
        initialize_model = false,
    )

directory ="./simulation_folder/"
build!(problem, output_dir = directory)

┌ Info: Overriding time_series_cache_size because time series is stored in memory
└ @ PowerSimulations /home/cju/.julia/packages/PowerSimulations/HKwUO/src/core/settings.jl:43


BuildStatus.BUILT = 0

In [24]:
optimization_container = PSI.get_optimization_container(problem)
@show typeof(optimization_container)

typeof(optimization_container) = PowerSimulations.OptimizationContainer


PowerSimulations.OptimizationContainer

We can access a lot of properties, see [here](https://github.com/NREL-SIIP/PowerSimulations.jl/blob/17891d4a61d071ce98efd96706ec7586e5d833cc/src/core/optimization_container.jl).

In [58]:
# get the jump model
optc_jump_model = PSI.get_jump_model(optimization_container)
# @show optc_jump_model

# get the objective function
optc_obj_func = PSI.get_objective_function(optimization_container)
# @show optc_obj_func

# get the variables
optc_vars = PSI.get_variables(optimization_container)
# @show optc_vars

@show typeof(ActivePowerVariable()) # https://github.com/NREL-SIIP/PowerSimulations.jl/blob/a2ae3da0d7692741ecf85e6d6f453a0e1bc44d4e/src/core/variables.jl
@show typeof(HydroEnergyReservoir)
var_h = PSI.get_variable(optimization_container, ActivePowerVariable(), HydroEnergyReservoir)

println("")

# get constraints
optc_cons = PSI.get_constraints(optimization_container)
optc_cons_keys = PSI.get_constraint_keys(optimization_container)
@show get(optc_cons, optc_cons_keys[1], "")
# @show optc_cons

typeof(ActivePowerVariable()) = ActivePowerVariable
typeof(HydroEnergyReservoir) = DataType

get(optc_cons, optc_cons_keys[1], "") = 2-dimensional DenseAxisArray{ConstraintRef,2,...} with index sets:
    Dimension 1, ["Solitude", "Park City", "Alta", "Brighton", "Sundance"]
    Dimension 2, 1:24
And data, a 5×24 Matrix{ConstraintRef}:
 ActivePowerVariable_ThermalStandard_{Solitude, 1} - 0.175 PieceWiseLinearCostVariable_Solitude_{pwl_2, 1} - 0.35 PieceWiseLinearCostVariable_Solitude_{pwl_3, 1} = 0.0  ActivePowerVariable_ThermalStandard_{Solitude, 2} - 0.175 PieceWiseLinearCostVariable_Solitude_{pwl_2, 2} - 0.35 PieceWiseLinearCostVariable_Solitude_{pwl_3, 2} = 0.0  ActivePowerVariable_ThermalStandard_{Solitude, 3} - 0.175 PieceWiseLinearCostVariable_Solitude_{pwl_2, 3} - 0.35 PieceWiseLinearCostVariable_Solitude_{pwl_3, 3} = 0.0  ActivePowerVariable_ThermalStandard_{Solitude, 4} - 0.175 PieceWiseLinearCostVariable_Solitude_{pwl_2, 4} - 0.35 PieceWiseLinearCostVariable_Solitude_{pwl_3, 

2-dimensional DenseAxisArray{ConstraintRef,2,...} with index sets:
    Dimension 1, ["Solitude", "Park City", "Alta", "Brighton", "Sundance"]
    Dimension 2, 1:24
And data, a 5×24 Matrix{ConstraintRef}:
 ActivePowerVariable_ThermalStandard_{Solitude, 1} - 0.175 PieceWiseLinearCostVariable_Solitude_{pwl_2, 1} - 0.35 PieceWiseLinearCostVariable_Solitude_{pwl_3, 1} = 0.0  …  ActivePowerVariable_ThermalStandard_{Solitude, 24} - 0.175 PieceWiseLinearCostVariable_Solitude_{pwl_2, 24} - 0.35 PieceWiseLinearCostVariable_Solitude_{pwl_3, 24} = 0.0
 ActivePowerVariable_ThermalStandard_{Park City, 1} - 0.11 PieceWiseLinearCostVariable_Park City_{pwl_2, 1} = 0.0                                                           ActivePowerVariable_ThermalStandard_{Park City, 24} - 0.11 PieceWiseLinearCostVariable_Park City_{pwl_2, 24} = 0.0
 ActivePowerVariable_ThermalStandard_{Alta, 1} - 0.026000000000000002 PieceWiseLinearCostVariable_Alta_{pwl_2, 1} = 0.0                                                

### End of my investigation

In [158]:
hyd[1].operation_cost

ThreePartCost(VariableCost{Vector{Tuple{Float64, Float64}}}([(0.0, 0.0), (525.0, 17.5), (1137.5, 35.0)]), 0.0, 0.0, 0.0)

In [124]:
ts = get_time_series(Deterministic, hyd[1], "max_active_power");

In [127]:
ts.single_time_series.data

336×1 TimeArray{Float64, 1, DateTime, Vector{Float64}} 2020-01-01T00:00:00 to 2020-01-14T23:00:00
│                     │ A      │
├─────────────────────┼────────┤
│ 2020-01-01T00:00:00 │ 0.0633 │
│ 2020-01-01T01:00:00 │ 0.0631 │
│ 2020-01-01T02:00:00 │ 0.0629 │
│ 2020-01-01T03:00:00 │ 0.0627 │
│ 2020-01-01T04:00:00 │ 0.0625 │
│ 2020-01-01T05:00:00 │ 0.0623 │
│ 2020-01-01T06:00:00 │ 0.0621 │
│ 2020-01-01T07:00:00 │ 0.0619 │
│ 2020-01-01T08:00:00 │ 0.0616 │
│ 2020-01-01T09:00:00 │ 0.0614 │
│ 2020-01-01T10:00:00 │ 0.0612 │
│ ⋮                   │ ⋮      │
│ 2020-01-14T14:00:00 │ 0.0709 │
│ 2020-01-14T15:00:00 │ 0.0669 │
│ 2020-01-14T16:00:00 │ 0.0662 │
│ 2020-01-14T17:00:00 │ 0.0669 │
│ 2020-01-14T18:00:00 │ 0.0737 │
│ 2020-01-14T19:00:00 │ 0.0735 │
│ 2020-01-14T20:00:00 │ 0.0652 │
│ 2020-01-14T21:00:00 │ 0.0693 │
│ 2020-01-14T22:00:00 │ 0.0731 │
│ 2020-01-14T23:00:00 │ 0.0666 │

In [79]:
ts = get_time_series(SingleTimeSeries, loads[1], "max_active_power");

In [91]:
length(get_initial_timestamp(ts):Hour(1):get_initial_timestamp(ts)+Hour(335))

336

In [81]:
get_time_series_timestamps(ts)

LoadError: MethodError: no method matching get_time_series_timestamps(::SingleTimeSeries)
[0mClosest candidates are:
[0m  get_time_series_timestamps(::InfrastructureSystems.InfrastructureSystemsComponent, [91m::Forecast[39m) at /Users/sdalvi/.julia/packages/InfrastructureSystems/2yQRJ/src/component.jl:264
[0m  get_time_series_timestamps(::InfrastructureSystems.InfrastructureSystemsComponent, [91m::Forecast[39m, [91m::Union{Nothing, DateTime}[39m; len) at /Users/sdalvi/.julia/packages/InfrastructureSystems/2yQRJ/src/component.jl:264
[0m  get_time_series_timestamps(::InfrastructureSystems.InfrastructureSystemsComponent, [91m::StaticTimeSeries[39m) at /Users/sdalvi/.julia/packages/InfrastructureSystems/2yQRJ/src/component.jl:278
[0m  ...

In [177]:
system = PSB.build_system(SIIPExampleSystems, "5_bus_hydro_uc_sys"; force_build =true);
remove_time_series!(system, DeterministicSingleTimeSeries)
for load in get_components(PowerLoad, system)
    ts = get_time_series(SingleTimeSeries, load, "max_active_power");
    ts_val = get_time_series_values(SingleTimeSeries, load, "max_active_power")
    time_stamps = get_initial_timestamp(ts):Hour(1):get_initial_timestamp(ts)+Hour(335)
    remove_time_series!(system, SingleTimeSeries, load, "max_active_power")
    data = TimeArray(time_stamps, vcat(ts_val, ts_val + rand(168)*0.2))
    forecast = SingleTimeSeries("max_active_power", data;
#         normalization_factor = Float64(maximum(data)),
#         scaling_factor_multiplier = PSY.get_max_active_power
        )
    add_time_series!(system, load, forecast)
end

for hyd in get_components(HydroDispatch, system), label in ["max_active_power"]
    ts = get_time_series(SingleTimeSeries, hyd, label);
    ts_val = get_time_series_values(SingleTimeSeries, hyd, label)
    time_stamps = get_initial_timestamp(ts):Hour(1):get_initial_timestamp(ts)+Hour(335)
    remove_time_series!(system, SingleTimeSeries, hyd, label)
    data = TimeArray(time_stamps, vcat(ts_val, ts_val + rand(168)*0.01))
    forecast = SingleTimeSeries(label, data;
#         normalization_factor = Float64(maximum(data)),
#         scaling_factor_multiplier = PSY.get_max_active_power
        )
    add_time_series!(system, hyd, forecast)
end

labels = ["max_active_power", "inflow"]
for hyd in get_components(HydroEnergyReservoir, system), label in labels
    ts = get_time_series(SingleTimeSeries, hyd, label);
    ts_val = get_time_series_values(SingleTimeSeries, hyd, label)
    time_stamps = get_initial_timestamp(ts):Hour(1):get_initial_timestamp(ts)+Hour(335)
    remove_time_series!(system, SingleTimeSeries, hyd, label)
    data = TimeArray(time_stamps, vcat(ts_val, ts_val + rand(168)*0.01))
    forecast = SingleTimeSeries(label, data;
#         normalization_factor = Float64(maximum(data)),
#         scaling_factor_multiplier = PSY.get_max_active_power
        )
    add_time_series!(system, hyd, forecast)
end
PSY.transform_single_time_series!(system, 168, Hour(168))

new_cost= Dict(
    "Alta" => ThreePartCost((0.0, 14.0), 0.0, 4.0, 2.0),
    "Park City" => ThreePartCost((0.0, 15.0), 0.0, 1.5, 0.75),
    "Solitude" => ThreePartCost((0.0, 30.0), 0.0, 3.0, 1.5),
    "Sundance" => ThreePartCost((0.0, 40.0), 0.0, 4.0, 2.0),
    "Brighton" => ThreePartCost((0.0, 10.0), 0.0, 1.5, 0.75),
    )
for th in get_components(ThermalStandard, system)
    set_operation_cost!(th, new_cost[th.name])
end

┌ Info: Building new system 5_bus_hydro_uc_sys from raw data
│   sys_descriptor.raw_data = /Users/sdalvi/.julia/packages/PowerSystemCaseBuilder/dPZHk/data
└ @ PowerSystemCaseBuilder /Users/sdalvi/.julia/packages/PowerSystemCaseBuilder/dPZHk/src/build_system.jl:35
┌ Info: Parsing csv data in Hydro_Upstream_Input.csv ...
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/tMGbo/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed Hydro_Upstream_Input.csv
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/tMGbo/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in branch.csv ...
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/tMGbo/src/parsers/power_system_table_data.jl:143
┌ Info: Successfully parsed branch.csv
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/tMGbo/src/parsers/power_system_table_data.jl:148
┌ Info: Parsing csv data in bus.csv ...
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/tMGbo/sr

In [178]:
PSY.to_json(system, "5bus_hydro_2week.json"; force=true)

┌ Info: Serialized time series data to 5bus_hydro_2week_time_series_storage.h5.
└ @ InfrastructureSystems /Users/sdalvi/.julia/packages/InfrastructureSystems/2yQRJ/src/time_series_storage.jl:65
┌ Info: Serialized System to 5bus_hydro_2week.json
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/tMGbo/src/base.jl:254


In [35]:
IS.add_time_series!(sys, component, forecast)

3600000 milliseconds

Probabilistic time_series (7):
   name: max_active_power
   data: DataStructures.SortedDict(DateTime("2020-01-01T00:00:00") => [0.1693830378615661 0.1693830378615661; 0.1553614870689571 0.1553614870689571; … ; 0.19801103804125686 0.19801103804125686; 0.18181861695641918 0.18181861695641918], DateTime("2020-01-02T00:00:00") => [0.1693830378615661 0.1693830378615661; 0.1553614870689571 0.1553614870689571; … ; 0.19801103804125686 0.19801103804125686; 0.18181861695641918 0.18181861695641918], DateTime("2020-01-03T00:00:00") => [0.1693830378615661 0.1693830378615661; 0.1553614870689571 0.1553614870689571; … ; 0.19801103804125686 0.19801103804125686; 0.18181861695641918 0.18181861695641918], DateTime("2020-01-04T00:00:00") => [0.1693830378615661 0.1693830378615661; 0.1553614870689571 0.1553614870689571; … ; 0.19801103804125686 0.19801103804125686; 0.18181861695641918 0.18181861695641918], DateTime("2020-01-05T00:00:00") => [0.1693830378615661 0.1693830378615661; 0.1553614870689571 0.15536148

In [61]:
budget_data

Dict{Any, Any} with 2 entries:
  "HydroDispatch1" => 2.82843
  "HydroDispatch3" => 2.82843

PSY.transform_single_time_series!(system, 48, Hour(24))

In [195]:
budget_data = Dict()
for h in  get_components(HydroEnergyReservoir, system)
    name = PSY.get_name(h)
    budget_data[name] = PSY.get_rating(h)
end

In [62]:
pwd()

"/Users/sdalvi/Documents/GitRepos/ProgressiveHedging.jl/examples"

In [175]:
template_ed = OperationsProblemTemplate(CopperPlatePowerModel)
set_device_model!(template_ed, ThermalStandard, ThermalDispatchNoMin)
set_device_model!(template_ed, PowerLoad, StaticPowerLoad)
# set_device_model!(template_ed, HydroDispatch, FixedOutput)
# set_device_model!(template_ed, HydroEnergyReservoir, HydroDispatchRunOfRiver)

problem = OperationsProblem(
    template_ed, 
    system, 
#     optimizer = solver, 
    balance_slack_variables = true,
    warm_start = true,
    export_pwl_vars = true,
        horizon =2,
    initial_time = DateTime("2020-01-08T00:00:00")
)
build!(problem, output_dir = ".")

┌ Info: Overriding time_series_cache_size because time series is stored in memory
└ @ PowerSimulations /Users/sdalvi/.julia/packages/PowerSimulations/0onip/src/core/settings.jl:41


BuildStatus.BUILT = 0

In [176]:
open("model.txt", "w") do f
    println(f, problem.internal.optimization_container.JuMPmodel)
end

In [201]:
function build_operations_problem(
        system::PSY.System,
        initial_time,
        network = CopperPlatePowerModel,
        directory ="./simulation_folder/"
    )
    template_ed = OperationsProblemTemplate(CopperPlatePowerModel)
    set_device_model!(template_ed, ThermalStandard, ThermalDispatchNoMin)
    set_device_model!(template_ed, PowerLoad, StaticPowerLoad)
    set_device_model!(template_ed, HydroDispatch, FixedOutput)
    set_device_model!(template_ed, HydroEnergyReservoir, HydroDispatchRunOfRiver)

    problem = OperationsProblem(
        template_ed, 
        system, 
        optimizer = solver, 
        balance_slack_variables = true,
        warm_start = true,
        export_pwl_vars = true,
        horizon =2,
        initial_time = initial_time #DateTime("2020-01-01T00:00:00")
    )
    build!(problem, output_dir = directory)
    return problem
end
    
function populate_ext!(problem::PSI.OperationsProblem, scenario_no, budget_data, total_scenarios)
    problem.ext["scenario_no"] = scenario_no
    problem.ext["budget_data"] = budget_data
    problem.ext["total_scenarios"] = total_scenarios
    return
end

populate_ext! (generic function with 1 method)

In [202]:
function populate_PH_model!(problem::PSI.OperationsProblem)
    no_scenarios = problem.ext["total_scenarios"]
    scenario_no = problem.ext["scenario_no"]
    budget_data = problem.ext["budget_data"]
    
    optimization_container = PSI.get_optimization_container(problem)
    time_steps = PSI.model_time_steps(optimization_container)
    var_h = PSI.get_variable(optimization_container, :P__HydroEnergyReservoir)
    
    constraint_budget = PSI.add_cons_container!(
        optimization_container,
        :Hydro_Budget_HydroEnergyReservoir,
        collect(keys(budget_data)),
    )
    constraint_scenarios = PSI.add_cons_container!(
        optimization_container,
        :PH_scenario_HydroEnergyReservoir,
        collect(keys(budget_data)),
    )
    budget_var = PSI.add_var_container!(
        optimization_container,
        :Pb_HydroEnergyReservoir,
        collect(keys(budget_data)),
        1:no_scenarios,
    )
    jump_model = PSI.get_jump_model(optimization_container)
    for (name, budget) in budget_data
        for sec in 1:no_scenarios
            budget_var[name, sec] = JuMP.@variable(
                optimization_container.JuMPmodel,
                base_name = "Pb_HydroEnergyReservoir_{$(name)_{$(sec)}}",
            )
        end
        constraint_budget[name] =
            JuMP.@constraint(jump_model, sum(var_h[name, t] for t in time_steps) <= budget_var[name, scenario_no+1])
        constraint_scenarios[name] = 
            JuMP.@constraint(jump_model, sum(budget_var[name, sec] for sec in 1:no_scenarios ) == budget)
        
    end
    return
end   

populate_PH_model! (generic function with 1 method)

In [203]:
function get_subproblem(problem::PSI.OperationsProblem)
    optimization_container = PSI.get_optimization_container(problem)
    jump_model = PSI.get_jump_model(optimization_container)
    
    first_stge_var =  JuMP.VariableRef[]
    secound_stge_var = JuMP.VariableRef[]
    for (name, cont) in PSI.get_variables(problem) 
        if name == :Pb_HydroEnergyReservoir
            push!(first_stge_var, cont...)
        else
            push!(secound_stge_var, cont...)
        end
    end
    variable_map = Dict{PH.StageID, Vector{JuMP.VariableRef}}(
        [
            PH.stid(1) => first_stge_var,
            PH.stid(2) => secound_stge_var,
        ])
#     vdict[PH.stid(1)] =  first_stge_var
#     vdict[PH.stid(2)] =  secound_stge_var
    return jump_model, variable_map
end
    

get_subproblem (generic function with 1 method)

In [204]:
function create_model(
    scenario_id::PH.ScenarioID,
    budget_data;
    kwargs...
)
    initial_time = [DateTime("2020-01-01T00:00:00"), DateTime("2020-01-08T00:00:00")]
    system = PSY.System("5bus_hydro_2week.json")
    problem = build_operations_problem(
        system,
        initial_time[PH.value(scenario_id) + 1],
        CopperPlatePowerModel,
        "./simulation_folder/"
    )
    populate_ext!(problem, PH.value(scenario_id), budget_data, 2)
    populate_PH_model!(problem)
    jump_model, variable_map = get_subproblem(problem)
    return JuMPSubproblem(jump_model, scenario_id, variable_map)
end

create_model (generic function with 1 method)

In [205]:
function build_scen_tree()
    probs = [0.5, 0.5]
    tree = PH.ScenarioTree()
    for l in 1:2
        PH.add_leaf(tree, tree.root, probs[l])
    end
    return tree
end

build_scen_tree (generic function with 1 method)

### Extensive Form Solve

In [206]:
ef_model = @time PH.solve_extensive(PH.two_stage_tree(2),
    create_model, 
    ()->Xpress.Optimizer(),
    budget_data,
)
# println(ef_model);

┌ Info: Deserializing with InMemoryTimeSeriesStorage is currently not supported. Using HDF
└ @ InfrastructureSystems /Users/sdalvi/.julia/packages/InfrastructureSystems/2yQRJ/src/system_data.jl:576
┌ Info: Loaded time series from storage file existing=5bus_hydro_2week_time_series_storage.h5 new=/var/folders/rt/vtygs7_52f55zj8d_8k_0kp93bgbsz/T/jl_mRVcze
└ @ InfrastructureSystems /Users/sdalvi/.julia/packages/InfrastructureSystems/2yQRJ/src/hdf5_time_series_storage.jl:84
┌ Info: Deserializing with InMemoryTimeSeriesStorage is currently not supported. Using HDF
└ @ InfrastructureSystems /Users/sdalvi/.julia/packages/InfrastructureSystems/2yQRJ/src/system_data.jl:576


FICO Xpress v8.8.0, Hyper, solve started 12:33:24, May 12, 2021
Heap usage: 106KB (peak 106KB, 4222KB system)
Minimizing LP 
Original problem has:
        64 rows           40 cols          140 elements
Presolved problem has:
         4 rows           32 cols           32 elements
Presolve finished in 0 seconds
Heap usage: 104KB (peak 162KB, 4224KB system)

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 1.00e+00,  1.00e+00] / [ 1.00e+00,  1.00e+00]
  RHS and bounds [min,max] : [ 3.16e-03,  2.00e+00] / [ 2.60e-02,  2.00e+00]
  Objective      [min,max] : [ 7.50e+00,  5.00e+05] / [ 7.50e+00,  5.00e+05]

 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0           .000000      D      4     0        .000000     0
     4          3.439920      D      0     0        .000000     0
Uncrunching matrix
Optimal solution found
Dual solved problem
  4 simplex iterations in 0s

Final objective                       :

┌ Info: Loaded time series from storage file existing=5bus_hydro_2week_time_series_storage.h5 new=/var/folders/rt/vtygs7_52f55zj8d_8k_0kp93bgbsz/T/jl_O2IJAo
└ @ InfrastructureSystems /Users/sdalvi/.julia/packages/InfrastructureSystems/2yQRJ/src/hdf5_time_series_storage.jl:84


In [207]:
open("model.txt", "w") do f
    println(f, ef_model)
end

In [208]:
println(JuMP.termination_status(ef_model))
println(JuMP.objective_value(ef_model))
for var in JuMP.all_variables(ef_model)
    println("$var = $(JuMP.value(var))")
end

OPTIMAL
3.439920274199271
P__ThermalStandard_{Solitude, 1}_{0} = 0.0
P__ThermalStandard_{Park City, 1}_{0} = 0.0
P__ThermalStandard_{Alta, 1}_{0} = 0.0
P__ThermalStandard_{Brighton, 1}_{0} = 0.0
P__ThermalStandard_{Sundance, 1}_{0} = 0.0
P__ThermalStandard_{Solitude, 2}_{0} = 0.0
P__ThermalStandard_{Park City, 2}_{0} = 0.0
P__ThermalStandard_{Alta, 2}_{0} = 0.0
P__ThermalStandard_{Brighton, 2}_{0} = 0.0
P__ThermalStandard_{Sundance, 2}_{0} = 0.0
P__HydroEnergyReservoir_{HydroDispatch1, 1}_{0} = 0.0
P__HydroEnergyReservoir_{HydroDispatch3, 1}_{0} = 0.10182419419756578
P__HydroEnergyReservoir_{HydroDispatch1, 2}_{0} = 0.0
P__HydroEnergyReservoir_{HydroDispatch3, 2}_{0} = 0.0933111098002867
γ⁺__P_{1, 1}_{0} = 0.0
γ⁺__P_{1, 2}_{0} = 0.0
γ⁻__P_{1, 1}_{0} = 0.0
γ⁻__P_{1, 2}_{0} = 0.0
Pb_HydroEnergyReservoir_{HydroDispatch1_{1}}_{0,1} = 0.0
Pb_HydroEnergyReservoir_{HydroDispatch3_{1}}_{0,1} = 0.1951353039978525
Pb_HydroEnergyReservoir_{HydroDispatch1_{2}}_{0,1} = 1.4142135623730951
Pb_HydroEn

In [209]:
st = PH.two_stage_tree(2)
(n, err, rerr, obj, soln, phd) = PH.solve(st,
                                          create_model,
                                          PH.ScalarPenaltyParameter(25.0), budget_data,
                                          atol=1e-8, rtol=1e-12, max_iter=500,
                                          report=10, # print residual info every 10 iterations
                                          )
println("Number of iterations: ", n)
println("L^2 error: ", err)
println(obj)

Initializing...
...launching workers...
...initializing subproblems...


┌ Info: Deserializing with InMemoryTimeSeriesStorage is currently not supported. Using HDF
└ @ InfrastructureSystems /Users/sdalvi/.julia/packages/InfrastructureSystems/2yQRJ/src/system_data.jl:576
┌ Info: Loaded time series from storage file existing=5bus_hydro_2week_time_series_storage.h5 new=/var/folders/rt/vtygs7_52f55zj8d_8k_0kp93bgbsz/T/jl_tjw3sL
└ @ InfrastructureSystems /Users/sdalvi/.julia/packages/InfrastructureSystems/2yQRJ/src/hdf5_time_series_storage.jl:84
┌ Info: Deserializing with InMemoryTimeSeriesStorage is currently not supported. Using HDF
└ @ InfrastructureSystems /Users/sdalvi/.julia/packages/InfrastructureSystems/2yQRJ/src/system_data.jl:576
┌ Info: Loaded time series from storage file existing=5bus_hydro_2week_time_series_storage.h5 new=/var/folders/rt/vtygs7_52f55zj8d_8k_0kp93bgbsz/T/jl_G6iexW
└ @ InfrastructureSystems /Users/sdalvi/.julia/packages/InfrastructureSystems/2yQRJ/src/hdf5_time_series_storage.jl:84


Solving...
Iter:    0   AbsR: 9.299037e-01   RelR: 1.254424e+00   Xhat: 7.075200e-01   X: 6.034370e-01
Iter:   10   AbsR: 2.386930e-08   RelR: 3.219928e-08   Xhat: 2.386930e-08   X: 1.091327e-14
Iter:   20   AbsR: 2.386919e-08   RelR: 3.219915e-08   Xhat: 2.386919e-08   X: 1.075542e-14
Iter:   30   AbsR: 2.386908e-08   RelR: 3.219902e-08   Xhat: 2.386908e-08   X: 1.075542e-14
Iter:   40   AbsR: 2.386897e-08   RelR: 3.219888e-08   Xhat: 2.386897e-08   X: 1.087480e-14
Iter:   50   AbsR: 2.386887e-08   RelR: 3.219875e-08   Xhat: 2.386887e-08   X: 1.091242e-14
Iter:   60   AbsR: 2.386876e-08   RelR: 3.219862e-08   Xhat: 2.386876e-08   X: 1.067663e-14
Iter:   70   AbsR: 2.386865e-08   RelR: 3.219849e-08   Xhat: 2.386865e-08   X: 1.063745e-14
Iter:   80   AbsR: 2.386854e-08   RelR: 3.219836e-08   Xhat: 2.386854e-08   X: 1.075542e-14
Iter:   90   AbsR: 2.386843e-08   RelR: 3.219823e-08   Xhat: 2.386843e-08   X: 1.091242e-14
Iter:  100   AbsR: 2.386833e-08   RelR: 3.219810e-08   Xhat: 2.386833

┌ Warning: Performed 500 iterations without convergence. Consider increasing max_iter from 500.
└ @ ProgressiveHedging /Users/sdalvi/.julia/packages/ProgressiveHedging/CyAni/src/algorithm.jl:374


 ──────────────────────────────────────────────────────────────────────────────
                                       Time                   Allocations      
                               ──────────────────────   ───────────────────────
       Tot / % measured:            5.84s / 97.9%            401MiB / 96.3%    

 Section               ncalls     time   %tot     avg     alloc   %tot      avg
 ──────────────────────────────────────────────────────────────────────────────
 Solution                   1    4.10s  71.8%   4.10s    246MiB  63.7%   246MiB
   Solve subproblems      500    2.91s  50.8%  5.81ms   42.7MiB  11.1%  87.5KiB
     Issuing solve ...    500    2.90s  50.7%  5.80ms   42.5MiB  11.0%  87.0KiB
     Other                500    658μs  0.01%  1.32μs     0.00B  0.00%    0.00B
   Finishing                1    199ms  3.49%   199ms   12.0MiB  3.10%  12.0MiB
   Update PH Vars         500   5.26ms  0.09%  10.5μs    562KiB  0.14%  1.12KiB
   User Callbacks         501    974μs 

In [210]:
aobj = PH.retrieve_aug_obj_value(phd)
println("Augmented Objective: ", aobj)
println("Difference: ", aobj - obj)

Augmented Objective: 3.4399213905889523
Difference: 1.7763568394002505e-15


In [211]:
soln

,variable,value,stage,scenarios
,String,Float64,Int64,String
1,Pb_HydroEnergyReservoir_{HydroDispatch1_{1}},0.707108,1,"0,1"
2,Pb_HydroEnergyReservoir_{HydroDispatch1_{2}},0.707105,1,"0,1"
3,Pb_HydroEnergyReservoir_{HydroDispatch3_{1}},0.672931,1,"0,1"
4,Pb_HydroEnergyReservoir_{HydroDispatch3_{2}},0.741283,1,"0,1"
5,"P__ThermalStandard_{Park City, 2}",9.52795e-11,2,0
6,"P__HydroEnergyReservoir_{HydroDispatch3, 1}",0.100985,2,0
7,"P__ThermalStandard_{Sundance, 1}",1.4855e-11,2,0
8,"P__ThermalStandard_{Alta, 2}",1.87179e-10,2,0
9,"γ⁺__P_{1, 2}",1.54142e-13,2,0


In [70]:
problem = build_operations_problem(
    system,
    DateTime("2020-01-01T00:00:00"),
    CopperPlatePowerModel,
    "./simulation_folder/"
);

┌ Info: Overriding time_series_cache_size because time series is stored in memory
└ @ PowerSimulations /Users/sdalvi/.julia/packages/PowerSimulations/0onip/src/core/settings.jl:41


In [71]:
v = problem.internal.optimization_container.variables

Dict{Symbol, AbstractArray} with 5 entries:
  :γ⁻__P                   => 2-dimensional DenseAxisArray{VariableRef,2,...} w…
  :P__ThermalStandard      => 2-dimensional DenseAxisArray{VariableRef,2,...} w…
  :P__HydroEnergyReservoir => 2-dimensional DenseAxisArray{VariableRef,2,...} w…
  :PWL_cost_vars           =>   [Park City, 2, 1]  =  {P__ThermalStandard_{Park…
  :γ⁺__P                   => 2-dimensional DenseAxisArray{VariableRef,2,...} w…

In [72]:
get_variables(problem)

Dict{Symbol, AbstractArray} with 5 entries:
  :γ⁻__P                   => 2-dimensional DenseAxisArray{VariableRef,2,...} w…
  :P__ThermalStandard      => 2-dimensional DenseAxisArray{VariableRef,2,...} w…
  :P__HydroEnergyReservoir => 2-dimensional DenseAxisArray{VariableRef,2,...} w…
  :PWL_cost_vars           =>   [Park City, 2, 1]  =  {P__ThermalStandard_{Park…
  :γ⁺__P                   => 2-dimensional DenseAxisArray{VariableRef,2,...} w…

In [73]:
populate_ext!(problem, 1, budget_data, 2)

In [74]:
populate_PH_model!(problem)

In [75]:
jump_model, variable_map = get_subproblem(problem);

In [76]:
variable_map

Dict{StageID, Vector{VariableRef}} with 2 entries:
  StageID(1) => [Pb_HydroEnergyReservoir_{HydroDispatch1_{1}}, Pb_HydroEnergyRe…
  StageID(2) => [γ⁻__P_{1, 1}, γ⁻__P_{1, 2}, P__ThermalStandard_{Solitude, 1}, …